In [2]:
!pip install torch torchdata
!pip install transformers datasets
!pip install evaluate rouge_score loralib peft
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8775d6de64bfa299cd463dd92755bc69d6a109128c409969cdb196605aae1686
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
from datasets import load_dataset
import transformers
import accelerate
import torch


print(transformers.__version__)
print(accelerate.__version__)



4.35.2
0.24.1


In [3]:
train_dataset = load_dataset("scientific_papers",'arxiv',split='train[:1%]')
val_data=load_dataset("scientific_papers",'arxiv',split='validation[:1%]')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [4]:
model_name='facebook/bart-large-cnn'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
def trainable_model_params(model):
  trainable_params=0
  all_params=0
  for _,param in model.named_parameters():
    all_params+=param.numel()
    if param.requires_grad:
      trainable_params+=param.numel()
  return f"trainable params: {trainable_params}\nall params: {all_params}\npercentage of trainable params: {100*trainable_params/all_params}"
print(trainable_model_params(original_model))

trainable params: 406290432
all params: 406290432
percentage of trainable params: 100.0


In [6]:
def tokenize_function(example):
    prompt = [article for article in example["article"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["abstract"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['article', 'abstract', 'section_names',])

Map:   0%|          | 0/2030 [00:00<?, ? examples/s]

In [11]:
tokenized_val_datasets = val_data.map(tokenize_function, batched=True)
tokenized_val_datasets = tokenized_val_datasets.remove_columns(['article', 'abstract', 'section_names',])

In [12]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets.shape}")
print(f"Validation: {tokenized_val_datasets.shape}")

Shapes of the datasets:
Training: (2030, 2)
Validation: (64, 2)


In [13]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="lora_only",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [14]:
peft_model=get_peft_model(original_model,lora_config)
print(trainable_model_params(peft_model))

trainable params: 1253376
all params: 407470080
percentage of trainable params: 0.30759951749095293


In [16]:
import time
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_val_datasets,
)

In [17]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
1,10.062500


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/vocab.json',
 './peft-dialogue-summary-checkpoint-local/merges.txt',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [8]:
text='A paragraph is defined as “a group of sentences or a single sentence that forms a unit” (Lunsford and Connors 116). Length and appearance do not determine whether a section in a paper is a paragraph. For instance, in some styles of writing, particularly journalistic styles, a paragraph can be just one sentence long.'

In [10]:
inputs=tokenizer(text,return_tensors='pt')
summary=tokenizer.decode(
    peft_model.generate(input_ids=inputs['input_ids'], max_new_tokens=200, temperature=1.2001, do_sample=True)[0],
    skip_special_tokens=True
    )

In [11]:
summary

'A paragraph is defined as “a group of sentences or a single sentence that forms a unit’ Length and appearance do not determine whether a section in a paper is a paragraph. For instance, in some styles of writing, particularly journalistic styles, a paragraph can be just one sentence long.'